# All Policies

Analyze the performance of our Whittle and Adaptive Policies

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_preferences')

In [4]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
from copy import deepcopy
import secrets
from itertools import combinations
from rmab.simulator import create_transitions_from_prob
from rmab.whittle_policies import whittle_index
from rmab.compute_whittle import arm_compute_whittle_multi_state, arm_compute_whittle, fast_arm_compute_whittle
from collections import Counter
import gurobipy as gp
from gurobipy import GRB

## Non RMAB-Version

In [5]:
N = 10
K = 5
probs = [random.random() for i in range(N)]
rewards = [random.random() for i in range(N)]
probs = np.array(probs)
rewards = np.array(rewards)

In [6]:
rewards

array([0.83325512, 0.65264265, 0.52739642, 0.74895119, 0.70772929,
       0.16214977, 0.3099193 , 0.16675392, 0.71938171, 0.39044687])

In [7]:
def run_trial(actions,probs,rewards):
    flips = [rewards[j] for j in range(len(probs)) if random.random() < probs[j] and actions[j]]
    if len(flips) > 0:
        return random.choice(flips)
    else:
        return 0

In [8]:
a = [probs[i] * rewards[i] for i in range(len(probs))]
b = [probs[i] for i in range(len(probs))]

In [9]:
a = [probs[i] * rewards[i] for i in range(len(probs))]
b = [probs[i] for i in range(len(probs))]
model = gp.Model("lfp")
x = model.addVars(len(probs), lb=0, name="x")
t = model.addVar(name="t")
model.setObjective(gp.quicksum(a[i] * x[i] for i in range(len(probs))), GRB.MAXIMIZE)
model.addConstr(gp.quicksum(b[i] * x[i] for i in range(len(probs))) == 1, name="c1")
model.addConstr(gp.quicksum(x[i] for i in range(len(probs))) == K*t, name="c2")
for i in range(len(probs)):
    model.addConstr(x[i] <= t, name=f"c3_{i}")
model.optimize()



Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-05
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 11 columns and 41 nonzeros
Model fingerprint: 0x02a8a48c
Coefficient statistics:
  Matrix range     [7e-03, 5e+00]
  Objective range  [4e-03, 6e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 12 rows, 11 columns, 41 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1911197e+30   1.530628e+31   4.382239e+00      0s
       6    7.4356448e-01   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.00 seconds (0.00 work units)
Optimal objective  7.435644764e-01


In [10]:
if model.status == GRB.OPTIMAL:
    y_opt = model.getAttr('x', x)
    t_opt = t.X
    action = [y_opt[i] / t_opt for i in range(N)]
    action = [round(i) for i in action]
    action = np.array(action)
    print(action)


[1 0 1 1 1 0 0 0 1 0]


In [11]:
trials = 1000
np.mean([run_trial(action,probs,rewards) for i in range(trials)])

0.611752194711202

## RMAB Version

In [12]:
transitions = np.zeros((N,3,2,3))

for i in range(N):
    transitions[i,0,0,1] = np.random.uniform(0,0.25)
    transitions[i,0,0,0] = 1-transitions[i,0,0,1]
    transitions[i,0,1,1] = np.random.uniform(transitions[i,0,0,1],1)
    transitions[i,1,0,1] = np.random.uniform(transitions[i,0,0,1],1)
    transitions[i,1,0,0] = 1-transitions[i,1,0,1]
    transitions[i,1,1,1] = np.random.uniform(max(transitions[i,0,1,1],transitions[i,1,0,1]),1)
    transitions[i,1,1,2] = np.random.uniform(0,1-transitions[i,1,1,1])
    transitions[i,0,1,2] = np.random.uniform(0,min(transitions[i,1,1,2],1-transitions[i,0,1,1]))
    transitions[i,2,0,0] = transitions[i,2,0,1] = 0
    transitions[i,2,1,1] = np.random.uniform(transitions[i,1,1,1],1)
    transitions[i,2,1,0] = 1-transitions[i,2,1,1]

    transitions[i,0,1,0] = 1-transitions[i,0,1,1]-transitions[i,0,1,2]
    transitions[i,1,1,0] = 1-transitions[i,1,1,1]-transitions[i,1,1,2]
    

In [13]:
curr_state = np.array([random.randint(0,1) for i in range(N)])
state_WI = []
state_WI_if_2 = []

for i in range(N):
    arm_transitions = transitions[i,:2,:,:2]
    for j in range(2):
        arm_transitions[j,1] /= np.sum(arm_transitions[j,1])
    state_WI.append(fast_arm_compute_whittle(arm_transitions[:,:,1], curr_state[i], 0.9, 1e-3,reward_function='combined',lamb=0.5,match_prob=0,num_arms=N))

# TODO: Incorporate the 3-state, multi-transition computation
for i in range(N):
    arm_transitions = np.zeros((2,2))
    arm_transitions[0,0] = transitions[i,0,0,1]
    arm_transitions[1,0] = transitions[i,2,0,1]
    arm_transitions[0,1] = transitions[i,0,1,1]
    arm_transitions[1,1] = transitions[i,2,1,1]
    state_WI_if_2.append(fast_arm_compute_whittle(arm_transitions, 1, 0.9, 1e-3,reward_function='combined',lamb=0.5,match_prob=0,num_arms=N)+1)
state_WI, state_WI_if_2

([0.014014218552536229,
  0.017544805734268823,
  0.01693453926666534,
  0.024383699406249484,
  0.024828738697437392,
  0.01154497561114773,
  0.015668457531609267,
  0.020500587019499927,
  0.013258255650057942,
  0.028503464014304623],
 [1.0275584083164615,
  1.036756023185565,
  1.0390327135515183,
  1.0361079434311378,
  1.0363185915692141,
  1.0323218142752209,
  1.0355574182324165,
  1.040044571138455,
  1.0410716262756303,
  1.0389555979264924])

In [14]:
a = [curr_state[i]*(state_WI_if_2[i]*transitions[i,curr_state[i],1,2] - state_WI[i]*transitions[i,curr_state[i],1,2]) for i in range(N)]
b = [transitions[i,curr_state[i],1,2] for i in range(N)]
c = [state_WI[i] for i in range(N)]
m = gp.Model()

# Create variables
z = m.addVars(N, lb=0, name="z")
t = m.addVar(lb=0, name="t")

# Constraint: sum(b[i] * z[i]) = t
# m.addConstr(gp.quicksum(b[i] * z[i] for i in range(N)) == t, "constraint_1")
m.addConstr(gp.quicksum(z[i] for i in range(N)) == K, "constraint_1")
for i in range(N):
    m.addConstr(z[i] <= 1, name=f"c3_{i}")

# Set objective
m.setObjective(gp.quicksum((a[i] + c[i]) * z[i] for i in range(N)), GRB.MAXIMIZE)

# Optimize the model
m.optimize()



Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 11 columns and 20 nonzeros
Model fingerprint: 0xd725ee30
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 10 rows and 1 columns
Presolve time: 0.00s
Presolved: 1 rows, 10 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2666829e+00   4.000000e+00   0.000000e+00      0s
       1    6.3918811e-01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.391881059e-01


In [16]:
if model.status == GRB.OPTIMAL:
    y_opt = model.getAttr('x', z)
    action = [y_opt[i] for i in range(N)]
    action = [round(i) for i in action]
    action = np.array(action)
    print(action)


[1 0 1 1 1 0 0 0 1 0]


In [87]:
a = [curr_state[i]*(state_WI_if_2[i]*transitions[i,curr_state[i],1,2] - state_WI[i]*transitions[i,curr_state[i],1,2]) for i in range(N)]
b = [transitions[i,curr_state[i],1,2] for i in range(N)]
c = [state_WI[i] for i in range(N)]
m = gp.Model()

# Create variables
z = m.addVars(N, lb=0, name="z")
t = m.addVar(lb=0, name="t")

# Constraint: sum(b[i] * z[i]) = t
m.addConstr(gp.quicksum(b[i] * z[i] for i in range(N)) == t, "constraint_1")
m.addConstr(gp.quicksum(z[i] for i in range(N)) == K*t, "constraint_1")
for i in range(N):
    m.addConstr(z[i] <= t, name=f"c3_{i}")

# Set objective
m.setObjective(gp.quicksum((a[i] + c[i]) * z[i] for i in range(N)), GRB.MAXIMIZE)

# Optimize the model
m.optimize()



Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 11 columns and 42 nonzeros
Model fingerprint: 0x6ad73930
Coefficient statistics:
  Matrix range     [1e-03, 5e+00]
  Objective range  [2e-02, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 12 rows, 11 columns, 42 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4882041e+29   1.196379e+31   8.488204e-01      0s
       5   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective -0.000000000e+00


In [88]:
if model.status == GRB.OPTIMAL:
    y_opt = model.getAttr('x', z)
    t_opt = t.X
    action = [min(y_opt[i],1) for i in range(N)]
    action = [round(i) for i in action]
    action = np.array(action)
    print(action)


[1 0 1 0 0 0 1 1 0 1]
